In [1]:
##CONNECTING TO snowflake
from snowflake.snowpark import Session
#from snowflake.core import Root
import toml 

toml_file_path = ('/Users/rpegu/Documents/Snowflake/MachineLearning/snowflakeml/configure.toml')
with open(toml_file_path,'r') as file:
    config_data = toml.load(file)
## read from toml file
user = config_data['RPEGU_AIML']['user']
pwd = config_data['RPEGU_AIML']['password']
account = config_data['RPEGU_AIML']['account']
role = config_data['RPEGU_AIML']['role']



connection_parameters = {
 "account": account,
 "user": user,
"password": pwd,
"role": role,  # optional
 }  

session = Session.builder.configs(connection_parameters).create()


In [2]:

session.sql("use warehouse DS_W").collect()
session.sql("use database  ML_MODELS").collect()
session.sql("use schema ML_MODELS.DS").collect()

from snowflake.snowpark.version import VERSION
import snowflake.snowpark.functions as F


snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


Connection Established with the following parameters:
User                        : RPEGU
Role                        : "ACCOUNTADMIN"
Database                    : "ML_MODELS"
Schema                      : "DS"
Warehouse                   : "DS_W"
Snowflake version           : 8.38.2
Snowpark for Python version : 1.22.1


In [3]:
from snowflake.ml.feature_store.examples.example_helper import ExampleHelper

example_helper = ExampleHelper(session, session.get_current_database(),'DS')
example_helper.list_examples().to_pandas()

,NAME,MODEL_CATEGORY,DESC,LABEL_COLS
0,new_york_taxi_features,regression,Features using taxi trip data trying to predic...,TOTAL_AMOUNT
1,airline_features,classification,Features using synthetic airline data to predi...,DEPARTING_DELAY
2,citibike_trip_features,regression,Features using citibike trip data trying to pr...,tripduration
3,wine_quality_features,regression,Features using wine quality data trying to pre...,quality


In [4]:
source_tables = example_helper.load_example('wine_quality_features')

In [5]:
for table in source_tables:
    print(f"{table}:")
    df = session.table(table).limit(5).to_pandas()
    print(df)

"ML_MODELS".DS.winedata:
   WINE_ID  FIXED_ACIDITY  VOLATILE_ACIDITY  CITRIC_ACID  RESIDUAL_SUGAR  \
0        1            7.4              0.70         0.00             1.9   
1        2            7.8              0.88         0.00             2.6   
2        3            7.8              0.76         0.04             2.3   
3        4           11.2              0.28         0.56             1.9   
4        5            7.4              0.70         0.00             1.9   

   CHLORIDES  FREE_SULFUR_DIOXIDE  TOTAL_SULFUR_DIOXIDE  DENSITY    PH  \
0      0.076                   11                    34   0.9978  3.51   
1      0.098                   25                    67   0.9968  3.20   
2      0.092                   15                    54   0.9970  3.26   
3      0.075                   17                    60   0.9980  3.16   
4      0.076                   11                    34   0.9978  3.51   

   SULPHATES  ALCOHOL  QUALITY  
0       0.56      9.4        5  
1      

## Create features with feature store
### Initialize Feature Store

In [6]:
from snowflake.ml.feature_store import (
    FeatureStore,
    FeatureView,
    Entity,
    CreationMode
)

fs = FeatureStore(
    session=session, 
    database=session.get_current_database(), 
    name=session.get_current_schema(), 
    default_warehouse=session.get_current_warehouse(),
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
)

/Users/rpegu/opt/anaconda3/envs/snowflake-ml/lib/python3.9/site-packages/snowflake/ml/feature_store/feature_store.py:302: UserWarning: The current snowflake-ml-python version out of date, package upgrade recommended (current=1.6.2, recommended>=1.6.3)
  self._check_feature_store_object_versions()


## Register Entities and Features View

In [7]:
## Register Entities and Features View

all_entities=[]
for e in example_helper.load_entities():
    entity =fs.register_entity(e)
    all_entities.append(entity)
fs.list_entities().show()

/Users/rpegu/opt/anaconda3/envs/snowflake-ml/lib/python3.9/site-packages/snowflake/ml/feature_store/feature_store.py:189: UserWarning: Entity WINE already exists. Skip registration.
  return f(self, *args, **kargs)


--------------------------------------------------
|"NAME"  |"JOIN_KEYS"  |"DESC"    |"OWNER"       |
--------------------------------------------------
|WINE    |["WINE_ID"]  |Wine ID.  |ACCOUNTADMIN  |
--------------------------------------------------



In [8]:
all_feature_views =[]
for fv in example_helper.load_draft_feature_views():
    rf = fs.register_feature_view(
        feature_view=fv,
        version='1.0'
    )
    all_feature_views.append(rf)

fs.list_feature_views().select('name','version','desc','refresh_freq').show()

/Users/rpegu/opt/anaconda3/envs/snowflake-ml/lib/python3.9/site-packages/snowflake/ml/feature_store/feature_store.py:544: UserWarning: FeatureView EXTRA_WINE_FEATURES/1.0 already exists. Skip registration. Set `overwrite` to True if you want to replace existing FeatureView.
  return self._get_feature_view_if_exists(feature_view.name, str(version))
/Users/rpegu/opt/anaconda3/envs/snowflake-ml/lib/python3.9/site-packages/snowflake/ml/feature_store/feature_store.py:544: UserWarning: FeatureView WINE_FEATURES/1.0 already exists. Skip registration. Set `overwrite` to True if you want to replace existing FeatureView.
  return self._get_feature_view_if_exists(feature_view.name, str(version))


---------------------------------------------------------------------------------------------------------
|"NAME"               |"VERSION"  |"DESC"                                              |"REFRESH_FREQ"  |
---------------------------------------------------------------------------------------------------------
|WINE_FEATURES        |1.0        |Managed features about wine quality which refre...  |1 day           |
|EXTRA_WINE_FEATURES  |1.0        |Static features about wine quality which never ...  |NULL            |
---------------------------------------------------------------------------------------------------------



## Check the Snowsight

In [9]:
label_cols = example_helper.get_label_cols()
timestamp_col = example_helper.get_training_data_timestamp_col()
excluded_cols = example_helper.get_excluded_cols()
join_keys=[key for entity in all_entities for key in entity.join_keys]
spine_table = example_helper.get_training_spine_table()

print(f'timestamp_cols:  {timestamp_col}')
print(f'label_col : {label_cols}')
print(f'excluded_cols: {excluded_cols}')
print(f'join_keys : {join_keys}')
print(f'spine table  : {spine_table}')

timestamp_cols:  None
label_col : ['QUALITY']
excluded_cols: ['WINE_ID']
join_keys : ['WINE_ID']
spine table  : "ML_MODELS".DS.winedata


In [10]:
source_df = session.sql(f"""
select {','.join(label_cols)},
       {','.join(join_keys)}
       {',' + timestamp_col if timestamp_col is not None else ''}  
from {spine_table}
  """)
spine_df = source_df.sample(n=1000)
spine_df.show()


-------------------------
|"QUALITY"  |"WINE_ID"  |
-------------------------
|5          |1          |
|5          |2          |
|5          |3          |
|6          |4          |
|5          |5          |
|7          |1178       |
|5          |7          |
|7          |1037       |
|7          |9          |
|5          |10         |
-------------------------



In [11]:
my_dataset = fs.generate_dataset(
    name = "wine_train_dataset",
    spine_df=spine_df,
    features = all_feature_views,
    spine_timestamp_col = timestamp_col,
    spine_label_cols=label_cols,
    exclude_columns=excluded_cols
)

In [14]:
training_data_df = my_dataset.read.to_snowpark_dataframe()
training_data_df.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"QUALITY"  |"SULPHATES"          |"ALCOHOL"           |"FIXED_ACIDITY"    |"CITRIC_ACID"         |"CHLORIDES"           |"TOTAL_SULFUR_DIOXIDE"  |"PH"                |"HYBRID_ACID"        |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|5          |0.5600000023841858   |9.399999618530273   |7.400000095367432  |0.0                   |0.07599999755620956   |34                      |3.509999990463257   |0.0                  |
|5          |0.6800000071525574   |9.800000190734863   |7.800000190734863  |0.0                   |0.09799999743700027   |67                      |3.200000047683716   |0.0                  |
|6          |0.6299999952316284   |10.3999996

## MODELLING with SNOWPARK ML



In [51]:
from snowflake.ml.modeling.ensemble import RandomForestRegressor
from snowflake.ml.modeling import metrics as snowml_metrics
from snowflake.snowpark.functions import abs as sp_abs, mean, col


def train_model_using_snowpark_ml(training_data_df):
    train, test = training_data_df.random_split([0.8, 0.2], seed=42)
    feature_columns = list(set(training_data_df.columns) - set(label_cols) - set(join_keys) - set([timestamp_col]))
    print(f"feature cols: {feature_columns}")
    
    rf = RandomForestRegressor(
        input_cols=feature_columns, label_cols=label_cols, 
        max_depth=3, n_estimators=20, random_state=42
    )

    rf.fit(train)
    predictions = rf.predict(test)

    output_label_names = ['OUTPUT_' + col for col in label_cols]
    mse = snowml_metrics.mean_squared_error(
        df=predictions, 
        y_true_col_names=label_cols, 
        y_pred_col_names=output_label_names
    )

    accuracy = 100 - snowml_metrics.mean_absolute_percentage_error(
        df=predictions,
        y_true_col_names=label_cols,
        y_pred_col_names=output_label_names
    )

    print(f"MSE: {mse}, Accuracy: {accuracy}")
    return rf, mse, accuracy



In [52]:
random_forest_model , mse, accuracy = train_model_using_snowpark_ml(training_data_df) 

feature cols: ['TOTAL_SULFUR_DIOXIDE', 'PH', 'SULPHATES', 'ALCOHOL', 'FIXED_ACIDITY', 'CITRIC_ACID', 'HYBRID_ACID', 'CHLORIDES']


/Users/rpegu/opt/anaconda3/envs/snowflake-ml/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/rpegu/opt/anaconda3/envs/snowflake-ml/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


MSE: 0.39308901508967226, Accuracy: 99.91063149649246


## Log the model Registry


In [53]:
print(mse,accuracy)

0.39308901508967226 99.91063149649246


In [54]:
from snowflake.ml.registry import Registry

reg = Registry(
    session=session, 
    database_name=session.get_current_database(), 
    schema_name=session.get_current_schema(),
)

In [55]:
model_name = "WINE_RANDOM_FOREST_REGRESSOR_MODEL"

mv = reg.log_model(
    model_name=model_name,
    model=random_forest_model,
    sample_input_data=training_data_df,
    comment="My model trained with feature views, dataset",
)

/Users/rpegu/opt/anaconda3/envs/snowflake-ml/lib/python3.9/contextlib.py:119: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  return next(self.gen)
/Users/rpegu/opt/anaconda3/envs/snowflake-ml/lib/python3.9/site-packages/snowflake/ml/model/_packager/model_packager.py:94: UserWarning: Inferring model signature from sample input or providing model signature for Snowpark ML Modeling model is not required. Model signature will automatically be inferred during fitting. 
  handler.save_model(


# USing the Model

In [27]:
reg.show_models()

,created_on,name,model_type,database_name,schema_name,comment,owner,default_version_name,versions,aliases
0,2024-10-10 07:40:45.518000-07:00,WINE_RANDOM_FOREST_REGRESSOR_MODEL,USER_MODEL,ML_MODELS,DS,None,ACCOUNTADMIN,V1,"[""SHAGGY_WOLVERINE_3"",""V1""]","{""DEFAULT"":""V1"",""FIRST"":""V1"",""LAST"":""SHAGGY_WO..."


In [28]:
m= reg.get_model('WINE_RANDOM_FOREST_REGRESSOR_MODEL')

In [29]:
m.show_versions()

,created_on,name,aliases,comment,database_name,schema_name,model_name,is_default_version,functions,metadata,user_data
0,2024-10-10 07:40:45.589000-07:00,V1,"[""DEFAULT"",""FIRST""]","My model trained with feature views, dataset",ML_MODELS,DS,WINE_RANDOM_FOREST_REGRESSOR_MODEL,true,"[""PREDICT""]",{},{}
1,2024-10-10 07:48:35.384000-07:00,SHAGGY_WOLVERINE_3,"[""LAST""]","My model trained with feature views, dataset",ML_MODELS,DS,WINE_RANDOM_FOREST_REGRESSOR_MODEL,false,"[""PREDICT""]",{},{}


In [31]:
m.default="SHAGGY_WOLVERINE_3"
m.show_versions()

,created_on,name,aliases,comment,database_name,schema_name,model_name,is_default_version,functions,metadata,user_data
0,2024-10-10 07:40:45.589000-07:00,V1,"[""FIRST""]","My model trained with feature views, dataset",ML_MODELS,DS,WINE_RANDOM_FOREST_REGRESSOR_MODEL,false,"[""PREDICT""]",{},{}
1,2024-10-10 07:48:35.384000-07:00,SHAGGY_WOLVERINE_3,"[""DEFAULT"",""LAST""]","My model trained with feature views, dataset",ML_MODELS,DS,WINE_RANDOM_FOREST_REGRESSOR_MODEL,true,"[""PREDICT""]",{},{}


## Inference

In [32]:
mv.show_functions()

[{'name': 'PREDICT',
  'target_method': 'predict',
  'target_method_function_type': 'FUNCTION',
  'signature': ModelSignature(
                      inputs=[
                          FeatureSpec(dtype=DataType.INT16, name='TOTAL_SULFUR_DIOXIDE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='PH'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='SULPHATES'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='ALCOHOL'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='FIXED_ACIDITY'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CITRIC_ACID'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='HYBRID_ACID'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='CHLORIDES')
                      ],
                      outputs=[
                          FeatureSpec(dtype=DataType.INT16, name='TOTAL_SULFUR_DIOXIDE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='PH'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='SULPHATES'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='ALCOHOL'),
  		FeatureSpec(dtype=DataType.DOUBL

In [34]:
## test data (random)
test_df  = training_data_df.sample(0.05)
test_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"QUALITY"  |"SULPHATES"         |"ALCOHOL"           |"FIXED_ACIDITY"    |"CITRIC_ACID"        |"CHLORIDES"          |"TOTAL_SULFUR_DIOXIDE"  |"PH"                |"HYBRID_ACID"       |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|5          |1.2000000476837158  |10.300000190734863  |8.100000381469727  |0.2199999988079071   |0.0689999982714653   |23                      |3.299999952316284   |1.781999945640564   |
|6          |0.6299999952316284  |10.199999809265137  |7.599999904632568  |0.15000000596046448  |0.10999999940395355  |73                      |3.1700000762939453  |1.1399999856948853  |
|7          |0.6800000071525574  |11.199999809265137  |7.19999980

In [35]:
remote_prediction = mv.run(test_df,function_name='predict')
remote_prediction.to_pandas()

,QUALITY,TOTAL_SULFUR_DIOXIDE,PH,SULPHATES,ALCOHOL,FIXED_ACIDITY,CITRIC_ACID,HYBRID_ACID,CHLORIDES,OUTPUT_QUALITY
0,5,58,3.46,0.72,10.200000,6.8,0.00,0.000,0.124,5.533058
1,5,68,3.46,0.63,9.500000,9.5,0.44,4.180,0.071,5.301522
2,4,13,3.41,0.57,11.000000,7.1,0.02,0.142,0.096,5.649061
3,6,10,3.35,0.60,9.700000,7.0,0.00,0.000,0.114,5.403947
4,5,59,3.30,0.46,9.400000,7.9,0.06,0.474,0.069,5.137498
5,6,19,3.11,0.62,10.800000,11.8,0.55,6.490,0.071,6.055647
6,5,145,3.16,0.88,9.200000,8.9,0.18,1.602,0.176,5.330785
7,5,33,3.36,0.45,9.400000,8.7,0.00,0.000,0.084,5.110069
8,6,37,3.46,0.57,10.600000,6.9,0.00,0.000,0.105,5.514652
9,6,26,3.67,0.56,11.600000,5.4,0.09,0.486,0.089,5.522976


In [56]:
mv.set_metric(metric_name="MSE",value=mse)
mv.set_metric(metric_name="Accuracy",value=accuracy)

In [58]:
mv.get_metric(metric_name="Accuracy")

99.91063149649246

In [59]:
@#lnode = mv.lineage(session=session, direction="upstream")

TypeError: (0000) lineage() got an unexpected keyword argument 'session'